In [9]:
from os import listdir
from os.path import isfile, join
import sys
sys.path.append("/Users/andyreagan/tools/python")
from kitchentable.dogtoys import *
from json import loads
from re import findall,UNICODE
from labMTsimple.labMTsimple.speedy import LabMT
my_LabMT = LabMT()
from labMTsimple.labMTsimple.storyLab import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from database.bookclass import Book_raw_data

import os
sys.path.append('/Users/andyreagan/projects/2014/09-books/database')
os.environ.setdefault('DJANGO_SETTINGS_MODULE','gutenbergdb.settings')
import django
django.setup()

from library.models import *

import pickle
from rdflib import Graph
import rdflib

import hashlib

loading LabMT with stopVal=0.0, for 10222 words


In [2]:
f = open("data/gutenberg/rdf-data-all.pickle","rb")
all_gut_metadata = pickle.loads(f.read())
f.close()

In [3]:
len(all_gut_metadata)

51163

### that's less than 51249!

In [4]:
# here are three specific things that I want
gut_author_name = rdflib.term.URIRef('http://www.gutenberg.org/2009/pgterms/name')
gut_lang = rdflib.term.URIRef('http://purl.org/dc/terms/language')
gut_title = rdflib.term.URIRef('http://purl.org/dc/terms/title')
gut_downloads = rdflib.term.URIRef('http://www.gutenberg.org/2009/pgterms/downloads')

stuff_i_want = (gut_author_name,gut_lang,gut_title,gut_downloads)
called = ("authors","lang_refs","title","downloads")

def read_rdf_2(g):
    parse_result = {"authors": [],
                    "lang_refs": [],
                    "title": [],
                    "downloads": [],}
    for stmt in g:
        if stmt[1] in stuff_i_want:
            ind = called[stuff_i_want.index(stmt[1])]
            parse_result[ind].append(stmt)
    # go find the bibref that I need
    parse_result["langs"] = ["" for i in range(len(parse_result["lang_refs"]))]
    for i,lang_ref in enumerate(parse_result["lang_refs"]):
        for stmt in g:
            if stmt[0] == lang_ref[2]:
                parse_result["langs"][i] = stmt
    return parse_result

In [5]:
if isfile("data/gutenberg/all-book-metadata-51249.p"):
    all_book_metadata = pickle.load(open("data/gutenberg/all-book-metadata-51249.p","rb"))
    all_book_metadata_raw = pickle.load(open("data/gutenberg/all-book-metadata-raw-51249.p","rb"))
else:
    # end_book = 20
    end_book = 51249
    all_book_metadata = [{} for i in range(end_book)]
    all_book_metadata_raw = [{} for i in range(end_book)]
    for i in range(end_book):
        i=i+1
        folder = join("data/gutenberg/cache/epub/",str(i))
        file = join(folder,"pg{0}.rdf".format(i))
        if i%500 == 0:
            print(file)
        g = Graph()
        g.parse(file, format="xml")
        p = read_rdf_2(g)
        all_book_metadata_raw[i-1] = p
        # stringify all of the entries
        p_str = {}
        for key in p:
            p_str[key] = list(map(lambda x: str(x[2]),p[key]))
        # print(p_str)
        all_book_metadata[i-1] = p_str
    pickle.dump(all_book_metadata,open("data/gutenberg/all-book-metadata-51249.p","wb"),pickle.HIGHEST_PROTOCOL)
    pickle.dump(all_book_metadata_raw,open("data/gutenberg/all-book-metadata-raw-51249.p","wb"),pickle.HIGHEST_PROTOCOL)

In [7]:
for i,m in enumerate(all_book_metadata):
    fname = "data/gutenberg/gutenberg-007/{0}.txt".format(i+1)
    
    if len(Book.objects.filter(gutenberg_id=(i+1))) > 0:
        continue
    
    if isfile(fname) and not isfile("data/gutenberg/gutenberg-008/{0}.p".format(i+1)):
        my_book_raw_data = Book_raw_data(filename=fname)
        my_word_hash = {}
        for word in my_book_raw_data.all_word_list:
            if word not in my_word_hash:
                my_word_hash[word] = 1
        pickle.dump(my_book_raw_data,open("data/gutenberg/gutenberg-008/{0}.p".format(i+1),"wb"))
    elif isfile("data/gutenberg/gutenberg-008/{0}.p".format(i+1)):
        my_book_raw_data = pickle.load(open("data/gutenberg/gutenberg-008/{0}.p".format(i+1),"rb"))
        my_word_hash = {}
        for word in my_book_raw_data.all_word_list:
            if word not in my_word_hash:
                my_word_hash[word] = 1
        
    my_authors = m["authors"]
    my_author_objects = []
    for j,author in enumerate(my_authors):
        # int(str(all_book_metadata_raw[0]["authors"][0][0]).split("/")[-1])
        if len(Author.objects.filter(fullname=author)) > 0:
            my_author_objects.append(Author.objects.filter(fullname=author)[0])
        else:
            author_id = int(str(all_book_metadata_raw[i]["authors"][j][0]).split("/")[-1])
            a = Author(fullname=author,gutenberg_id=author_id)
            a.save()
            my_author_objects.append(a)
    # many fields are optional....
    if len(m["title"]) > 0:
        my_title = m["title"][0]
    else:
        my_title = ""
    if len(m["downloads"]) > 0:
        my_downloads = int(m["downloads"][0])
    else:
        my_downloads = 0
    if len(m["langs"]) > 0:
        my_lang = m["langs"][0]
        my_lang_id = lang_lookup(my_lang)
    else:
        my_lang = "na" # short for NOT AVAILABLE
        my_lang_id = -1
    # if this title already in there...skip
    if len(Book.objects.filter(gutenberg_id=(i+1))) > 0:
        continue
    if isfile(fname):
        full_text = open(fname,"r").read()
        my_hash = hashlib.md5(full_text.encode("utf8")).hexdigest()
        my_book = Book(title=my_title,
                       pickle_object="data/gutenberg/gutenberg-008/{0}.p".format(i+1),
                       hash=my_hash,
                       # authors=my_author_objects,
                       language=my_lang,
                       lang_code_id=my_lang_id,
                       downloads=my_downloads,
                       from_gutenberg=True,
                       gutenberg_id=(i+1),
                       txt_file_path=fname,
                       length=len(my_book_raw_data.all_word_list),
                       numUniqWords=len(my_word_hash),
                       exclude=False
                    )
        my_book.save()
    else:
        my_hash = ""
        my_book = Book(title=my_title,
                       hash=my_hash,
                       # authors=my_author_objects,
                       language=my_lang,
                       lang_code_id=my_lang_id,
                       downloads=my_downloads,
                       from_gutenberg=True,
                       gutenberg_id=(i+1),
                       exclude=True,
                       excludeReason="No file."
                    )
        my_book.save()
    for a in my_author_objects:
        my_book.authors.add(a)
        my_book.save()

In [ ]:
all_book_metadata[0]

In [ ]:
int(str(all_book_metadata_raw[0]["authors"][0][0]).split("/")[-1])

In [ ]:
my_author_objects